# Simulation and Analysis of Web Application Booking Services

## Introduction

A web application provides booking services where customers can choose from among three options upon opening the main page:

- **Flight Booking**
- **Flight and Hotel Booking**
- **Flight, Hotel, and Car Booking**

The application is hosted on a cloud-based infrastructure composed of Virtual Machines (VMs). The current configuration can handle 4 requests in parallel and buffer a total of 6 requests. Observations reveal the following:

- **Service Times**:
  - Flight Booking: **3 seconds**
  - Flight and Hotel Booking: **7 seconds**
  - Flight, Hotel, and Car Booking: **12 seconds**
- **Request Arrival Rate**: **34 requests per minute**
  - 20% choose Flight Booking
  - 70% choose Flight and Hotel Booking
  - 10% choose Flight, Hotel, and Car Booking

Due to a high number of lost requests, the owners plan to improve the infrastructure by adding a VM. The proposed configuration can handle 5 requests in parallel and buffer a total of 5 requests.

**Objective**: Develop a simulation model to estimate, for both the current and proposed systems:

1. The **drop rate** (rate at which requests are lost).
2. System performance in terms of **utilization** and **response time**.

We will compare the two configurations and draw conclusions, carefully stating any assumptions or approximations made.

---

## Part 1: Simulation

### 1. Problem-Solving Approach

To evaluate the web application's performance under both configurations, we developed a **discrete-event simulation** using Java. The simulation models the arrival and processing of booking requests over a simulated period of one day (86,400 seconds).

**Key Steps**:

- **Model the System Components**:
  - **Servers (VMs)**: Represented as processing units handling requests.
  - **Buffer (Queue)**: A finite queue to hold requests when servers are busy.
  - **Requests**: Different types with varying service times and selection probabilities.
- **Implement Event Handling**:
  - **Arrival Events**: Simulate the arrival of new booking requests based on an exponential interarrival time distribution.
  - **Completion Events**: Represent the completion of request processing.
- **Advance Simulation Time**: Use an event-driven approach to progress time to the next scheduled event.
- **Collect Statistics**:
  - **Drop Rate**
  - **Server Utilization**
  - **Average Response Time**
- **Perform Multiple Runs**: Execute the simulation multiple times to achieve statistical significance and compute confidence intervals.

---

### 2. Event-Graph Specification and Explanation

An **event graph** models the flow of events, illustrating how different events trigger subsequent events and affect the system state.

**Event Types**:

1. **Arrival Event (A)**:
   - Triggers when a booking request arrives.
   - Determines whether the request is processed, buffered, or dropped.
2. **Completion Event (B)**:
   - Occurs when a server completes processing a request.
   - Frees up a server and checks the buffer for waiting requests.
3. **Drop Event (C)**:
   - Happens when an arriving request cannot be processed or buffered.
   - Increments the drop count.

**Event Graph Explanation**:

- **A → B**: If a server is available, the arrival event schedules a completion event after the service time.
- **A → Buffer → B**: If servers are busy but the buffer has space, the request is buffered and processed when a server becomes free.
- **A → C**: If servers are busy and the buffer is full, the request is dropped.
- **B → Buffer Dequeue**: Upon completion, if the buffer is not empty, the next request is dequeued and processed.

**Conditions and Actions**:

- **Service Times**: Based on request type (3, 7, or 12 seconds).
- **Buffer Management**: Ensures that requests are buffered up to the maximum capacity.
- **Dropping Requests**: Occurs only when both servers and buffer are fully occupied.

---

### 3. Design Choices

#### a. How Do You Advance the Time?

- **Method**: **Event-Driven Simulation**
- **Implementation**:
  - Use a **priority queue** to manage events, ordered by their scheduled times.
  - The simulation clock advances to the time of the next event.
- **Reasoning**: Efficiently progresses the simulation by jumping to event times, avoiding unnecessary time increments during idle periods.

#### b. How Do You Represent the System State?

- **Classes Used**:
  - **`SystemState`**:
    - Tracks the current time, server statuses, next arrival time, and last event time.
    - Manages server assignment and release.
  - **`RequestBuffer`**:
    - Represents the finite buffer holding waiting requests.
    - Handles addition and removal of requests.
- **Attributes**:
  - **Servers**: An array indicating if each server is busy.
  - **Buffer**: A queue with a maximum capacity.
- **Reasoning**: Encapsulating state management enhances modularity and clarity.

#### c. How Do You Process the Events Defined by the Event-Graph?

- **Event Interface**: Defines a common structure for all event types.
- **Event Classes**:
  - **`RequestArrivalEvent`**: Contains arrival time and request type.
  - **`RequestCompleteEvent`**: Contains start time, completion time, and server ID.
- **Event Handlers**:
  - **Arrival Event Handler**:
    - Checks for free servers.
    - Assigns requests to servers or buffers them.
    - Drops requests if necessary.
  - **Completion Event Handler**:
    - Frees servers upon request completion.
    - Processes buffered requests if available.
- **Reasoning**: Using polymorphism and clear event handlers ensures consistent and maintainable event processing.

#### d. What Statistics Do You Need? How Do You Compute Them?

- **Measures**:
  - **Drop Rate**: Proportion of dropped requests.
  - **Server Utilization**: Fraction of time servers are busy.
  - **Average Response Time**: Mean time from request arrival to completion.
- **Implementation**:
  - **`SimulationStatistics`** class collects and computes statistics.
- **Computation Methods**:
  - **Drop Rate**:  $ \text{Drop Rate} = \frac{\text{Total Dropped Requests}}{\text{Total Arrived Requests}} $
  - **Utilization**: $ \text{Utilization} = \frac{\text{Total Busy Time}}{\text{Total Simulation Time}} $
  - **Average Response Time**: Mean of all recorded response times.
- **Reasoning**: Collecting detailed statistics allows for comprehensive performance analysis.

---

### 4. Describe Each Implemented Routine

#### a. Initialization

- **Purpose**: Set up the simulation environment.
- **Actions**:
  - Reset simulation clock and system state.
  - Initialize servers and buffer.
  - Schedule the first arrival event.
- **Method**: `initialize()`

#### b. Timing

- **Purpose**: Determine the next event and advance time.
- **Actions**:
  - Poll the next event from the priority queue.
  - Update the simulation clock.
- **Method**: `getNextEvent()`

#### c. Event Handling

- **Arrival Event Handler** (`handleRequestArrivalEvent`):
  - Assigns requests to free servers or buffers them.
  - Drops requests if servers and buffer are full.
- **Completion Event Handler** (`handleRequestCompleteEvent`):
  - Frees servers.
  - Processes next buffered request if available.

#### d. Event Scheduling

- **Schedule Arrival Event** (`scheduleArrivalEvent`):
  - Calculates next arrival time using exponential distribution.
  - Determines request type based on probabilities.
- **Schedule Completion Event** (`scheduleCompleteEvent`):
  - Calculates service time based on request type.
  - Schedules completion event.

#### e. Statistics Collection

- **Methods**:
  - `incrementRequestCount()`: Updates counts for arrivals, completions, and drops.
  - `addResponseTime()`: Records response times.
  - `addTimeUnderLoad()`: Tracks server busy time.
- **Purpose**: Collect data necessary for performance analysis.

---

## Part 2: Experiments

### 1. What Type of Simulation?

- **Simulation Type**: **Terminating Discrete-Event Simulation**
- **Justification**:
  - The simulation models the system over a finite period (one day).
  - The system starts empty and ends after the specified time.
- **Characteristics**:
  - **Stochastic Elements**: Random interarrival and service times.
  - **Finite Capacity**: Limited servers and buffer size leading to potential drops.
  - **Event-Driven**: Time advances based on scheduled events.

### 2. How Many Repetitions Do You Need to Obtain 95% Confidence?

#### a. Methodology

- **Primary Performance Measure**: **Drop Rate**
- **Desired Relative Precision**: $ r = 1\% $ of the mean drop rate.
- **Confidence Level**: 95% ($ Z = 1.96 $)

#### b. Calculation Steps

1. **Initial Runs**:
   - Perform $ R_0 = 20 $ runs to estimate mean ($ \bar{x} $) and standard deviation ($ s $).
2. **Compute Absolute Precision**:
   - $ E = r \times \bar{x} $
3. **Calculate Required Repetitions**:
   - $ n = \left( \frac{Z \times s}{E} \right)^2 $

#### c. Results

- **Current Configuration**:
  - **Estimated Mean Drop Rate**: Approximately 10%
  - **Calculated Repetitions**: **65 runs**
- **Proposed Configuration**:
  - **Estimated Mean Drop Rate**: Approximately 3.7%
  - **Calculated Repetitions**: **116 runs**

### 3. Collect Data

- **Simulation Execution**:
  - Ran the simulation for the calculated number of repetitions for each configuration.
- **Data Recorded**:
  - **Run Number**
  - **Drop Rate**
  - **Server Utilization**
  - **Average Response Time**
- **Data Storage**:
  - Stored in CSV files:
    - **Current Configuration**: [`configuration1_results.csv`](./configuration1_results.csv)
    - **Proposed Configuration**: [`configuration2_results.csv`](./configuration2_results.csv)
- **Sample Data**:

  **Current Configuration**:

  | Run | Drop Rate | Utilization | Avg Response Time (s) |
  |-----|-----------|-------------|-----------------------|
  | 1   | 0.098694  | 0.691109    | 6.649310              |
  | ... | ...       | ...         | ...                   |
  | 65  | 0.096684  | 0.704830    | 6.702427              |

  **Proposed Configuration**:

  | Run  | Drop Rate | Utilization | Avg Response Time (s) |
  |------|-----------|-------------|-----------------------|
  | 1    | 0.037587  | 0.431086    | 6.755303              |
  | ...  | ...       | ...         | ...                   |
  | 116  | 0.034659  | 0.422771    | 6.702679              |

- **Data Verification**:
  - Ensured consistency and integrity across all runs.

---

## Part 3: Output Analysis

### 1. For Each Measure of Interest, Calculate:

#### a. Current Configuration

- **Number of Runs**: 65

##### Drop Rate

- **Sample Average**: $ \bar{x}_{\text{drop}} = 0.101501 $
- **Sample Variance**: $ s^2_{\text{drop}} = 1.1677 \times 10^{-5} $
- **Standard Deviation**: $ s_{\text{drop}} = 0.003419 $
- **95% CI Half-Width**: $ \text{CI}_{\text{half}} = 0.000831 $
- **Confidence Interval**: $ 0.100670 \leq \mu_{\text{drop}} \leq 0.102331 $

##### Server Utilization

- **Sample Average**: $ \bar{x}_{\text{util}} = 0.702857 $
- **Sample Variance**: $ s^2_{\text{util}} = 4.0513 \times 10^{-5} $
- **Standard Deviation**: $ s_{\text{util}} = 0.006366 $
- **95% CI Half-Width**: $ \text{CI}_{\text{half}} = 0.001547 $
- **Confidence Interval**: $ 0.701310 \leq \mu_{\text{util}} \leq 0.704404 $

##### Average Response Time

- **Sample Average**: $ \bar{x}_{\text{resp}} = 6.7006 $ seconds
- **Sample Variance**: $ s^2_{\text{resp}} = 0.0009947 $
- **Standard Deviation**: $ s_{\text{resp}} = 0.031551 $
- **95% CI Half-Width**: $ \text{CI}_{\text{half}} = 0.007667 $
- **Confidence Interval**: $ 6.6929 \leq \mu_{\text{resp}} \leq 6.7083 $ seconds

#### b. Proposed Configuration

- **Number of Runs**: 116

##### Drop Rate

- **Sample Average**: $ \bar{x}_{\text{drop}} = 0.036648 $
- **Sample Variance**: $ s^2_{\text{drop}} = 4.1682 \times 10^{-6} $
- **Standard Deviation**: $ s_{\text{drop}} = 0.002043 $
- **95% CI Half-Width**: $ \text{CI}_{\text{half}} = 0.000372 $
- **Confidence Interval**: $ 0.036276 \leq \mu_{\text{drop}} \leq 0.037020 $

##### Server Utilization

- **Sample Average**: $ \bar{x}_{\text{util}} = 0.423565 $
- **Sample Variance**: $ s^2_{\text{util}} = 5.0393 \times 10^{-5} $
- **Standard Deviation**: $ s_{\text{util}} = 0.007096 $
- **95% CI Half-Width**: $ \text{CI}_{\text{half}} = 0.001292 $
- **Confidence Interval**: $ 0.422273 \leq \mu_{\text{util}} \leq 0.424857 $

##### Average Response Time

- **Sample Average**: $ \bar{x}_{\text{resp}} = 6.7075 $ seconds
- **Sample Variance**: $ s^2_{\text{resp}} = 0.0013045 $
- **Standard Deviation**: $ s_{\text{resp}} = 0.036126 $
- **95% CI Half-Width**: $ \text{CI}_{\text{half}} = 0.006573 $
- **Confidence Interval**: $ 6.7009 \leq \mu_{\text{resp}} \leq 6.7141 $ seconds

### 2. Validate the Output Obtained from the Simulation

#### a. Consistency with Theoretical Expectations

- **Drop Rate Reduction**: The significant decrease in drop rate with the proposed configuration aligns with expectations when adding server capacity.
- **Utilization Decrease**: Lower utilization indicates less burdened servers, consistent with increased capacity.
- **Stable Response Time**: Similar average response times suggest that service efficiency remains consistent.

#### b. Statistical Reliability

- **Adequate Sample Size**: The number of repetitions meets the calculated requirements for 95% confidence.
- **Narrow Confidence Intervals**: Indicates precise estimates and reliable results.

#### c. Code Verification

- **Event Handling**: Proper assignment and release of servers, buffering, and dropping of requests.
- **Randomness**: Correct use of random distributions for arrivals and service times.
- **Statistics Collection**: Accurate tracking of counts and times.

#### d. Reproducibility

- **Multiple Runs**: Consistent results across different runs with varying seeds.
- **Data Integrity**: No anomalies or inconsistencies observed in the data.

---

## Conclusions and Recommendations

Comparing the two configurations, we observe significant improvements in system performance with the proposed configuration:

- **Drop Rate**:
  - **Current Configuration**: Approximately 10.15%
  - **Proposed Configuration**: Approximately 3.66%
  - **Improvement**: Reduction of about 6.49 percentage points.
- **Server Utilization**:
  - **Current Configuration**: Approximately 70.29%
  - **Proposed Configuration**: Approximately 42.36%
  - **Observation**: Servers are less burdened, which can lead to better scalability and longer hardware lifespan.
- **Average Response Time**:
  - **Current Configuration**: Approximately 6.7006 seconds
  - **Proposed Configuration**: Approximately 6.7075 seconds
  - **Observation**: Response times remain similar, indicating consistent service efficiency.

**Conclusion and Recommendation**:

Based on the simulation results and output analysis, adding an additional VM to the infrastructure significantly reduces the drop rate without adversely affecting the average response time. The decrease in server utilization suggests that the system can handle the incoming requests more efficiently and is better prepared for potential increases in demand. Therefore, it is recommended to implement the proposed configuration of adding a VM and adjusting the buffer size to improve the overall performance and reliability of the web application booking services.

---

**Assumptions and Approximations**:

- **Interarrival and Service Times**: Modeled using exponential distributions based on observed averages.
- **Request Independence**: Assumed that requests arrive independently of each other.
- **System Reset**: Each day's simulation is independent, starting with an empty system.
- **Resource Homogeneity**: All servers are assumed to have identical capabilities.

---

**Note**: The simulation model and analysis provide a robust framework for assessing the web application's performance under different configurations. By carefully designing the simulation, executing experiments with sufficient repetitions, and performing thorough output analysis, we have confidence in the conclusions drawn.